# Predykcja energii słonecznej, wytwarzanej przez panele fotowoltaiczne

Katarzyna Boczek 117317

## Biblioteki
Do wykonania zadania wykorzystano następujące biblioteki:

In [1]:
import pandas as pd
import numpy as np
import time

from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor

## Wczytanie zbiorów danych
W pierwszym kroku wczytano zbiór uczący oraz zbiór testowy.

In [2]:
train_df = pd.read_csv("train.csv", sep=",")
test_df = pd.read_csv("test.csv", sep=",")

## Przetworzenie wartości pustych
Ilość wytwarzanej energii jest uzależniona od promieniowania słonecznego. Jeśli ilość wytworznej energii jest większa od zera (`kwh > 0`) a wartość promieniowania słonecznego jest zerowa (`irradiamento = 0`) to należy uznać to za wartość brakującą. Wartości te uzupełniono średnią wartością promieniowania o danej godzinie w danym dniu.

In [3]:
missing = train_df[(train_df["irradiamento"] == 0) & (train_df["kwh"] > 0)]
occurring = train_df[~train_df.isin(missing)].dropna()

def set_value(row):
    similar_data = train_df[train_df["data"] == row["data"]]
    row["irradiamento"] = np.mean(similar_data["irradiamento"])
    return row

missing = missing.apply(set_value, axis=1)

train_df = occurring.append(missing)

## Przygotowanie zbioru uczącego i testowego
Model oraz predykcję energii oparto o pięć zmiennych najbardziej skorelowanych z wytwarzaną energią.

In [4]:
x_train = train_df.loc[:, ["irradiamento", "irri", "irr_pvgis_mod", "humidity", "azimuthi"]]
y_train = train_df["kwh"]

x_test = test_df.loc[:, ["irradiamento", "irri", "irr_pvgis_mod", "humidity", "azimuthi"]]

## Stworzenie i nauczenie modelu
Przetestowano różne modele, np. model regresji liniowej, aproksymację wielomianową czy drzewo decyzyjne. Ostatecznie zdecydowano się na model Random Forest w połączeniu z baggingiem mimo, iż nie osiągnął on najlepszych rezultatów w publicznym rankingu konkursu. Na podstawie testów przeprowadzonych na zbiorze walidacyjnym stwierdzono, że model ten w ogólności osiąga lepsze wyniki. Ustawienie parametru `random_state = 3` zapewnia powtarzalność wyników.

In [6]:
regressor = BaggingRegressor(base_estimator=RandomForestRegressor(random_state=3), random_state=3)
regressor.fit(x_train, y_train)

BaggingRegressor(base_estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=3, verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=3, verbose=0, warm_start=False)

## Predykcja
Posiadając nauczony model przystąpiono do predykcji wytworzonej energii.

In [7]:
y_test = regressor.predict(x_test)

## Przygotowanie wyników do zapisu
Przed zapisaniem wyników odpowiednio je sformatowano. Do identyfikatorów pomiarów doklejono przewidzianą dla nich ilość wytworzonej energii. Dodano również odpowiedni nagłówek.

In [8]:
results_df = pd.DataFrame(test_df["id"])
results_df["kwh"] = y_test
results_df.columns = ["Id", "kwh"]

## Zapis wyników do pliku
Wyniki zapisano do pliku `csv`, który w nazwie posiada timestamp wygenerowania tegoż wyniku.

In [9]:
results_filename = "results_" + time.strftime("%Y%m%d-%H%M%S") + ".csv"
results_df.to_csv(results_filename, index=False)